In [8]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn.functional as F
import torchvision.transforms as T
from torch import Tensor

from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torchvision.utils import make_grid
from torchvision.transforms.functional import to_pil_image
from PIL import Image

import os

print(os.getcwd()) # dosya yolunu ver
%run ../Model.ipynb
%run ../Dataset.ipynb

%run ./Flow_Model.ipynb

plt.rcParams['font.size'] = 14
plt.rcParams['font.family'] = 'DeJavu Serif'
plt.rcParams['font.serif'] = ['Times New Roman']

/arf/home/tunal/ondemand/PhD Thesis Starting/01_SON/Tik-4/Tez/08-CelebA/05-UAE_Latent_FM


In [9]:
# Hyper-Parameters & Settings
batch_size = 500

In [10]:
# Veri yolu
root_dir = '../data/img_align_celeba'

train_dataset = CelebADataset(img_dir=root_dir, attr_path='../data/list_attr_celeba.csv', partition_path='../data/list_eval_partition.csv' , mode='train')
val_dataset   = CelebADataset(img_dir=root_dir, attr_path='../data/list_attr_celeba.csv', partition_path='../data/list_eval_partition.csv', mode='val')
test_dataset  = CelebADataset(img_dir=root_dir, attr_path='../data/list_attr_celeba.csv', partition_path='../data/list_eval_partition.csv', mode='test')

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=len(val_dataset))
test_loader  = DataLoader(test_dataset, batch_size=len(test_dataset))

In [12]:
# UAE modelini çalıştır

path = '../ty/results/UAE_CelebA_HQ'
model = torch.load(path + '.model', weights_only=False, map_location='cpu')
model.eval()

To_Uniform(
  (encoder): SimpleEncoder(
    (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv4): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (bn4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv5): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (bn5): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (fc): Sequential(
      (0): Linear(in_features=8192, out_features=2048, bias=True)
      (1): SiLU

In [13]:
# "results" klasörünü oluştur (zaten varsa hata vermez)
os.makedirs("results", exist_ok=True)

flow_name = './results/UAE_Latent_FM'
flow = Flow()

optimizer = torch.optim.Adam(flow.parameters(), 0.0003)
loss_fn = nn.MSELoss()

In [14]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# --- 1. Veriyi encode et ---
def get_encoded_loader(data_loader, encoder, device='cpu'):
    encoder.eval()
    encoded = []

    with torch.no_grad():
        for x in data_loader:
            x = x.to(device)
            z = encoder(x)
            encoded.append(z.cpu())

    all_z = torch.cat(encoded, dim=0)
    return torch.utils.data.DataLoader(all_z, batch_size=data_loader.batch_size, shuffle=True)

# --- 2. Encode edilmiş veri ile loader oluştur ---
encoded_train_loader = get_encoded_loader(train_loader, model.encoder, device=device)
encoded_val_loader = get_encoded_loader(val_loader, model.encoder, device=device)

In [ ]:
# --- 3. Flow Matching eğitimi ---
train_cost = []
val_cost = []
best_val_loss = float('inf')

for epoch in range(1000):
    epoch_losses = []

    flow.train()
    for z1 in encoded_train_loader:
        z1 = z1.to(device)  # encode edilmiş temsil (batch_size x latent_dim)
        z0 = torch.rand_like(z1)
        t = torch.rand(len(z1), 1, device=device)

        zt = (1 - t) * z0 + t * z1
        dzt = z1 - z0

        optimizer.zero_grad()
        pred = flow(t=t, x_t=zt)
        loss = loss_fn(pred, dzt)
        loss.backward()
        optimizer.step()

        epoch_losses.append(loss.item())

    mean_train_loss = sum(epoch_losses) / len(epoch_losses)
    train_cost.append(mean_train_loss)

    # Validation
    flow.eval()
    val_losses = []
    with torch.no_grad():
        for z1_val in encoded_val_loader:
            z1_val = z1_val.to(device)
            z0_val = torch.rand_like(z1_val)
            t_val = torch.rand(len(z1_val), 1, device=device)

            zt_val = (1 - t_val) * z0_val + t_val * z1_val
            dzt_val = z1_val - z0_val

            pred_val = flow(t=t_val, x_t=zt_val)
            val_loss = loss_fn(pred_val, dzt_val)
            val_losses.append(val_loss.item())

    mean_val_loss = sum(val_losses) / len(val_losses)
    val_cost.append(mean_val_loss)

    # En iyi modeli kaydet
    if mean_val_loss < best_val_loss:
        print("Saved!")
        best_val_loss = mean_val_loss
        torch.save(flow, flow_name + '.model')

    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/1000 - Train Loss: {mean_train_loss:.4f} - Val Loss: {mean_val_loss:.4f}")

# Son modeli kaydet
torch.save(flow, flow_name + '_final.model')

Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Epoch 10/1000 - Train Loss: 0.1121 - Val Loss: 0.1116
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Epoch 20/1000 - Train Loss: 0.1083 - Val Loss: 0.1080
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Epoch 30/1000 - Train Loss: 0.1057 - Val Loss: 0.1053
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Epoch 40/1000 - Train Loss: 0.1039 - Val Loss: 0.1035
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Epoch 50/1000 - Train Loss: 0.1025 - Val Loss: 0.1024
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Epoch 60/1000 - Train Loss: 0.1013 - Val Loss: 0.1012
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Epoch 70/1000 - Train Loss: 0.1004 - Val Loss: 0.1002
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Epoch 80/1000 - Train Loss: 0.0997 - Val Loss: 0.0994
Saved!
Saved!
Saved!
Saved!
Saved!
Epoch 90/1000 - Train Loss: 0.0989 - Val Loss: 0.0988
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Epo

In [ ]:
# CSV dosyasına kaydet
train_losses = train_cost  # liste veya numpy array
val_losses = val_cost

np.savetxt("results/losses.csv", 
           np.column_stack((train_losses, val_losses)), 
           delimiter=",", 
           header="train_loss,val_loss", 
           comments="")

In [ ]:
# Burada Eğitimin içinde veri encode ediliyor, bu yüzden çok yavaş çalışıyor
"""
train_cost = []
val_cost = []
best_val_loss = float('inf')  # Başlangıçta çok büyük bir değer

for epoch in range(1000):
    epoch_losses = []

    flow.train()
    for x in train_loader:

        x_1 = model.encoder(x)  # encode edilmiş temsil (batch_size x latent_dim)
        
        x_0 = torch.rand_like(x_1)
        t = torch.rand(len(x_1), 1)

        x_t = (1 - t) * x_0 + t * x_1
        dx_t = x_1 - x_0

        optimizer.zero_grad()
        pred = flow(t=t, x_t=x_t)
        loss = loss_fn(pred, dx_t)
        loss.backward()
        optimizer.step()

        epoch_losses.append(loss.item())

    mean_train_loss = sum(epoch_losses) / len(epoch_losses)
    train_cost.append(mean_train_loss)

    # Validation
    flow.eval()
    val_losses = []
    with torch.no_grad():
        for x_val in val_loader:
            x_1_val = model.encoder(x_val) # encode edilmiş temsil (batch_size x latent_dim)
            
            x_0_val = torch.rand_like(x_1_val)
            t_val = torch.rand(len(x_1_val), 1)

            x_t_val = (1 - t_val) * x_0_val + t_val * x_1_val
            dx_t_val = x_1_val - x_0_val

            pred_val = flow(t=t_val, x_t=x_t_val)
            val_loss = loss_fn(pred_val, dx_t_val)
            val_losses.append(val_loss.item())

    mean_val_loss = sum(val_losses) / len(val_losses)
    val_cost.append(mean_val_loss)

    # En iyi modeli val loss'a göre kaydet
    if mean_val_loss < best_val_loss:
        print('Saved!')
        best_val_loss = mean_val_loss
        torch.save(flow, flow_name + '.model')

    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/1000 - Train Loss: {mean_train_loss:.4f} - Val Loss: {mean_val_loss:.4f}")

# Son epoch modelini ayrı istersen:
torch.save(flow, flow_name + '_final.model')
"""